<a href="https://colab.research.google.com/github/ta269uec/semantic-search/blob/develop/k_means_sentence_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing all the packages necessary + loading the data

In [2]:
!pip install datasets transformers torch sentence-transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 19.7 MB/s eta 0:0

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
LIB_LOCATION = "/content/drive/MyDrive/git-repos/semantic-search/"
OUT_DATA_LOCATION = LIB_LOCATION + "data/"

In [4]:
# Clone if you do not have this.
#!git clone -b develop https://github.com/ta269uec/semantic-search.git /content/drive/MyDrive/git-repos/semantic-search/

In [4]:
import sys
sys.path.append(LIB_LOCATION)

In [5]:
from src.sentence_encoder import SentenceEncoder
from src.sst2 import get_reviews_sst2
from src.kmeans import cluster_k_means

In [6]:
pos_sentences, neg_sentences = get_reviews_sst2()

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset sst2 downloaded and prepared to /root/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5. Subsequent calls will reuse this data.


Filter:   0%|          | 0/67349 [00:00<?, ? examples/s]

Filter:   0%|          | 0/67349 [00:00<?, ? examples/s]

In [7]:
st = SentenceEncoder()

In [8]:
pos_embeddings, neg_embeddings = st.encode(pos_sentences), st.encode(neg_sentences)

# Build K-Means Clusters

In [9]:
N = 250
clustered_pos_sentences, clustered_pos_embeddings = cluster_k_means(corpus = pos_sentences,\
                                                            corpus_embeddings = pos_embeddings.cpu(),\
                                                            num_clusters = N)
clustered_neg_sentences, clustered_neg_embeddings = cluster_k_means(corpus = neg_sentences,\
                                                            corpus_embeddings = neg_embeddings.cpu(),\
                                                            num_clusters = N)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
def build_clusters(N, pos_sentences, pos_embeddings, neg_sentences, neg_embeddings):
  clustered_pos_sentences, clustered_pos_embeddings = cluster_k_means(corpus = pos_sentences,\
                                                            corpus_embeddings = pos_embeddings.cpu(),\
                                                            num_clusters = N)
  clustered_neg_sentences, clustered_neg_embeddings = cluster_k_means(corpus = neg_sentences,\
                                                            corpus_embeddings = neg_embeddings.cpu(),\
                                                            num_clusters = N)
  return (clustered_pos_sentences, clustered_pos_embeddings), (clustered_neg_sentences, clustered_neg_embeddings)


In [ ]:
import os
import pickle
def serialize_clusters(root, ds_name, label, n_clusters, results):
  subdir = f"{ds_name}_{label}_{n_clusters}"
  path_results = os.path.join(root, subdir)
  if not os.path.exists(path_results):
    os.mkdir(path_results)
  sentences, embeddings = results[0], results[1]
  with open(os.path.join(path_results, "sentences.pkl"), 'wb') as handle:
    pickle.dump(sentences, handle, protocol=pickle.HIGHEST_PROTOCOL)
  with open(os.path.join(path_results, "embeddings.pkl"), 'wb') as handle:
    pickle.dump(sentences, handle, protocol=pickle.HIGHEST_PROTOCOL)    
  return


In [ ]:
root = OUT_DATA_LOCATION
ds_name = "sst2"
N = 10
res_pos, res_neg = build_clusters(N, pos_sentences, pos_embeddings, neg_sentences, neg_embeddings)
serialize_clusters(root, ds_name, label="pos", n_clusters=N, results=res_pos)

## Sanity Check 1: Are clusters semantically similar?

In [52]:
clusters = [len(clustered_pos_embeddings[i]) for i in range(len(clustered_pos_embeddings))]

In [53]:
import torch
X = torch.stack(clustered_pos_embeddings[0])

In [54]:
from sentence_transformers import util

cos_scores = util.cos_sim(X, X).numpy()

In [55]:
cos_scores

array([[1.        , 0.56275344, 0.16633949, ..., 0.34173852, 0.33133203,
        0.33750123],
       [0.56275344, 1.0000001 , 0.25965416, ..., 0.31524807, 0.38929945,
        0.3176574 ],
       [0.16633949, 0.25965416, 0.9999997 , ..., 0.18074846, 0.2621564 ,
        0.18604575],
       ...,
       [0.34173855, 0.315248  , 0.18074849, ..., 1.0000002 , 0.377027  ,
        0.32030904],
       [0.33133203, 0.38929948, 0.26215637, ..., 0.37702698, 1.        ,
        0.29153726],
       [0.33750117, 0.31765735, 0.18604569, ..., 0.32030913, 0.29153726,
        0.99999976]], dtype=float32)